In [1]:
# CÀI ĐẶT & CẤU HÌNH (SETUP & CONFIGURATION)

# 0.1. Cài đặt các thư viện cần thiết
!pip install -q fpdf2 noisereduce librosa tensorflow scikit-learn matplotlib seaborn pytz PyDrive2

# 0.2. Import thư viện
import os
import glob
import random
import datetime
import pytz
import shutil
import joblib
import zipfile
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from fpdf import FPDF
from tqdm.notebook import tqdm
import librosa
import noisereduce as nr
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.metrics import classification_report, confusion_matrix
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from kaggle_secrets import UserSecretsClient
from oauth2client.service_account import ServiceAccountCredentials
from tensorflow.keras.regularizers import l2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.7/72.7 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.7/251.7 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 58.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 4.6 MB/s eta 0:00:00


2025-08-26 14:19:47.062722: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756217987.242812      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756217987.294228      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# THIẾT LẬP CẤU HÌNH
# 0.4. Thiết lập SEED để đảm bảo kết quả tái lặp
SEED = 42
def set_seed(seed_value):
    """Cố định seed cho các thư viện để đảm bảo kết quả nhất quán."""
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
    random.seed(seed_value)
    np.random.seed(seed_value)
    tf.random.set_seed(seed_value)
set_seed(SEED)

# 0.5. Thiết lập phông chữ (đã được đơn giản hóa)
font_path = None # Sẽ sử dụng font mặc định

# 0.6. Cấu hình các đường dẫn và tham số
# --- CẤU HÌNH KAGGLE DATASET (QUAN TRỌNG) ---
KAGGLE_PROCESSED_DATA_PATH = "/kaggle/input/ngt-spectrogram-id/" 

# --- CẤU HÌNH KẾT NỐI GOOGLE DRIVE (TÙY CHỌN, ĐỂ LƯU KẾT QUẢ) ---
DRIVE_RESULTS_FOLDER_ID = '13DW3yr_AVDDbu-Onv58LKLE3TaPuJRDq' 

# --- Đường dẫn trên máy ảo Kaggle (Không cần sửa) ---
KAGGLE_OUTPUT_PATH = "/kaggle/working/output_results"
os.makedirs(KAGGLE_OUTPUT_PATH, exist_ok=True)

# --- Các thông số cấu hình thử nghiệm ---
CLASSES_TO_TRAIN = ['covid', 'asthma', 'healthy', 'tuberculosis']
ALL_CLASSES = ['healthy', 'asthma', 'covid', 'tuberculosis']
N_SPLITS = 5
TEST_SPLIT_RATIO = 0.15
USE_DATA_AUGMENTATION = False
USE_FOCAL_LOSS = True
MODEL_ID = f'ResNet50V2_CV_{"_".join(CLASSES_TO_TRAIN)}'
EPOCHS = 50
BATCH_SIZE = 64
LEARNING_RATE = 3e-5
EARLY_STOPPING_PATIENCE = 7
MIN_DELTA = 1e-4
SHUFFLE_BUFFER_SIZE = 2048 

# Các tham số không đổi
SAMPLE_RATE = 16000
N_MELS = 128
N_FFT = 2048
HOP_LENGTH = 512
SILENCE_THRESHOLD_DB = 20
IMG_SIZE = (128, 157)
INPUT_SHAPE = (IMG_SIZE[0], IMG_SIZE[1], 3)

In [3]:
# KHỞI TẠO CÁC HÀM CẦN THIẾT

def get_patient_id(filepath, class_name):
    filename = os.path.basename(filepath)
    if class_name.lower() in ['asthma', 'covid', 'healthy']:
        return filename.split('_')[0]
    elif class_name.lower() == 'tuberculosis':
        return '_'.join(filename.split('_')[:-1]).replace('.npy', '')
    else:
        return filename.split('_')[0]
        
def parse_and_process(filepath, label_onehot):
    """Hàm đọc file, stack kênh, chuẩn hóa và làm sạch."""
    def _read_npy(path):
        return np.load(path.decode()).astype(np.float32)
    
    spec = tf.numpy_function(_read_npy, [filepath], tf.float32)
    spec = tf.stack([spec, spec, spec], axis=-1)
    spec.set_shape(INPUT_SHAPE)
    
    spec_shape = tf.shape(spec)
    spec_flat = tf.reshape(spec, (1, -1))
    
    def _scale(data):
        scaled_data = scaler.transform(data)
        # Luôn làm sạch dữ liệu ngay sau khi scale
        return np.nan_to_num(scaled_data)
    
    scaled_flat = tf.numpy_function(_scale, [spec_flat], tf.float32)
    spec_scaled = tf.reshape(scaled_flat, spec_shape)
    
    return spec_scaled, label_onehot

def augment(spectrogram, label):
    """Hàm áp dụng augmentation."""
    spectrogram = spec_augment(spectrogram)
    return spectrogram, label
    
def focal_loss_from_logits_optimized(active_indices, gamma=2.0, alpha=0.25):
    """
    Hàm Focal Loss phiên bản tối ưu, nhận active_indices từ bên ngoài.
    """
    def focal_loss_fixed(y_true, y_pred):
        y_true = tf.cast(y_true, 'float32')

        # Không cần tính lại active_indices, chỉ cần sử dụng
        y_true_filtered = tf.gather(y_true, active_indices, axis=-1)
        y_pred_filtered = tf.gather(y_pred, active_indices, axis=-1)

        # Phần còn lại giữ nguyên
        cross_entropy = tf.nn.softmax_cross_entropy_with_logits(
            labels=y_true_filtered, logits=y_pred_filtered
        )
        
        probs = tf.nn.softmax(y_pred_filtered)
        pt = tf.reduce_sum(y_true_filtered * probs, axis=-1)
        
        focal_term = (1.0 - pt) ** gamma
        loss = alpha * focal_term * cross_entropy
        return loss
        
    return focal_loss_fixed

# HÀM SPEC_AUGMENT ĐÃ SỬA LỖI
def spec_augment(spectrogram, time_masking_para=40, frequency_masking_para=15,
                 num_time_masks=1, num_freq_masks=1):
    """
    Hàm SpecAugment đã được sửa lỗi để làm việc với shape (freq, time, channels).
    """
    # Lấy ra các chiều để làm việc
    freq_bins = tf.shape(spectrogram)[0]
    time_steps = tf.shape(spectrogram)[1]
    
    spectrogram_aug = spectrogram

    # 1. Frequency Masking (che một khoảng tần số)
    for _ in range(num_freq_masks):
        f = tf.random.uniform(shape=(), minval=0, maxval=frequency_masking_para, dtype=tf.int32)
        f0 = tf.random.uniform(shape=(), minval=0, maxval=freq_bins - f, dtype=tf.int32)

        # Tạo một "mặt nạ" 1D cho chiều tần số
        freq_mask_1d = tf.concat([
            tf.ones(shape=(f0,), dtype=spectrogram.dtype),
            tf.zeros(shape=(f,), dtype=spectrogram.dtype),
            tf.ones(shape=(freq_bins - f0 - f,), dtype=spectrogram.dtype)
        ], axis=0)
        
        # Reshape mặt nạ để nó có thể nhân với ảnh phổ 3D
        # Shape sẽ là (freq, 1, 1) để broadcast qua chiều thời gian và kênh
        freq_mask_3d = tf.reshape(freq_mask_1d, (freq_bins, 1, 1))
        spectrogram_aug = spectrogram_aug * freq_mask_3d

    # 2. Time Masking (che một khoảng thời gian)
    for _ in range(num_time_masks):
        t = tf.random.uniform(shape=(), minval=0, maxval=time_masking_para, dtype=tf.int32)
        t0 = tf.random.uniform(shape=(), minval=0, maxval=time_steps - t, dtype=tf.int32)

        # Tạo một "mặt nạ" 1D cho chiều thời gian
        time_mask_1d = tf.concat([
            tf.ones(shape=(t0,), dtype=spectrogram.dtype),
            tf.zeros(shape=(t,), dtype=spectrogram.dtype),
            tf.ones(shape=(time_steps - t0 - t,), dtype=spectrogram.dtype)
        ], axis=0)

        # Reshape mặt nạ để nó có thể nhân với ảnh phổ 3D
        # Shape sẽ là (1, time, 1) để broadcast qua chiều tần số và kênh
        time_mask_3d = tf.reshape(time_mask_1d, (1, time_steps, 1))
        spectrogram_aug = spectrogram_aug * time_mask_3d
        
    return spectrogram_aug

def create_model(input_shape, num_classes):
    
    # 1. Khởi tạo mô hình
    base_model = ResNet50V2(weights='imagenet', include_top=False, input_shape=input_shape)
    
    # 2. Đóng băng các lớp ban đầu (ví dụ: 100 lớp đầu tiên)
    # ResNet-50V2 có khoảng 190 lớp. Đóng băng một nửa là mức khởi điểm tốt.
    FREEZE_UNTIL_LAYER = 100
    for layer in base_model.layers[:FREEZE_UNTIL_LAYER]:
        layer.trainable = False
        
    # 3. Đảm bảo phần còn lại của mô hình được huấn luyện
    # Cần set trainable = True cho base_model sau khi đóng băng các layer cụ thể
    base_model.trainable = True 
    
    # 4. Xây dựng lớp đầu ra (đã có L2 Regularization)
    inputs = Input(shape=input_shape)
    # training=True là cần thiết để đảm bảo các lớp Batch Normalization hoạt động đúng (cho phần không bị đóng băng)
    x = base_model(inputs, training=True) 
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    
    outputs = Dense(num_classes, 
                    activation='linear',
                    kernel_regularizer=l2(0.001)
                   )(x) 
    return Model(inputs, outputs)

def load_data_from_df(df):
    X, y = [], []
    for _, row in df.iterrows():
        X.append(np.load(row['filepath']))
        y.append(row['label'])
    return np.array(X), np.array(y)

def get_grad_cam(model, img_array, last_conv_layer_name, pred_index=None):
    grad_model = Model([model.inputs], [model.get_layer(last_conv_layer_name).output, model.output])
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(tf.cast(img_array, tf.float32))
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]
    grads = tape.gradient(class_channel, last_conv_layer_output)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

def overlay_grad_cam(spec, heatmap, alpha=0.6):
    heatmap_resized = tf.image.resize(heatmap[..., np.newaxis], (spec.shape[0], spec.shape[1]))
    heatmap_resized = np.uint8(255 * heatmap_resized)
    jet = plt.cm.get_cmap("jet")
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap_resized.squeeze()]
    spec_display = np.stack([spec]*3, axis=-1)
    spec_display = (spec_display - spec_display.min()) / (spec_display.max() - spec_display.min())
    superimposed_img = jet_heatmap * alpha + spec_display
    superimposed_img = np.clip(superimposed_img, 0, 1)
    return superimposed_img

class PDFReport(FPDF):
    def header(self):
        self.set_font('Arial', 'B', 12)
        self.cell(0, 10, 'BAO CAO KET QUA HUAN LUYEN MO HINH AI', 0, 1, 'C')
        self.ln(10)
    def footer(self):
        self.set_y(-15)
        self.set_font('Arial', 'I', 8)
        self.cell(0, 10, f'Trang {self.page_no()}', 0, 0, 'C')
    def chapter_title(self, title):
        self.set_font('Arial', 'B', 12)
        self.cell(0, 10, title, 0, 1, 'L')
        self.ln(5)
    def chapter_body(self, content):
        self.set_font('Arial', '', 10)
        safe_content = content.encode('latin-1', 'replace').decode('latin-1')
        self.multi_cell(0, 5, safe_content)
        self.ln()
    def add_image_section(self, title, img_path):
        self.chapter_title(title)
        if os.path.exists(img_path):
            self.image(img_path, x=None, y=None, w=180)
            self.ln(5)
        else:
            self.chapter_body(f"Khong tim thay hinh anh: {img_path}")

def authenticate_gdrive():
    user_secrets = UserSecretsClient()
    secret_value = user_secrets.get_secret("google_service_account_key")
    with open("service_account.json", "w") as f:
        f.write(secret_value)
    scope = ["https://www.googleapis.com/auth/drive"]
    gauth = GoogleAuth()
    gauth.credentials = ServiceAccountCredentials.from_json_keyfile_name("service_account.json", scope)
    drive = GoogleDrive(gauth)
    return drive

def upload_folder_to_drive(drive, folder_path, parent_folder_id):
    folder_name = os.path.basename(folder_path)
    print(f"Đang tạo thư mục '{folder_name}' trên Google Drive...")
    folder_metadata = {'title': folder_name, 'mimeType': 'application/vnd.google-apps.folder', 'parents': [{'id': parent_folder_id}]}
    folder = drive.CreateFile(folder_metadata)
    folder.Upload()
    
    print(f"Bắt đầu tải nội dung của '{folder_name}'...")
    for item in tqdm(os.listdir(folder_path), desc=f"Uploading {folder_name}"):
        item_path = os.path.join(folder_path, item)
        if os.path.isfile(item_path):
            gfile = drive.CreateFile({'title': item, 'parents': [{'id': folder['id']}]})
            gfile.SetContentFile(item_path)
            gfile.Upload(param={'supportsTeamDrives': True})
        elif os.path.isdir(item_path):
            upload_folder_to_drive(drive, item_path, folder['id'])

In [4]:
# CHUẨN BỊ DỮ LIỆU
print("Bắt đầu chuẩn bị và phân chia dữ liệu...")
all_files_to_split = []
for class_name in ALL_CLASSES:
    source_dir = os.path.join(KAGGLE_PROCESSED_DATA_PATH, class_name)
    if os.path.exists(source_dir):
        files = glob.glob(os.path.join(source_dir, '*.npy'))
        for f in files:
            all_files_to_split.append({'filepath': f, 'label': class_name})

all_data_df = pd.DataFrame(all_files_to_split)
all_data_df['patient_id'] = all_data_df.apply(lambda row: get_patient_id(row['filepath'], row['label']), axis=1)

print("Tách tập Test cuối cùng (Hold-out set)...")
patient_ids = all_data_df['patient_id'].unique()
np.random.shuffle(patient_ids)
test_patient_count = int(len(patient_ids) * TEST_SPLIT_RATIO)
test_patients = patient_ids[:test_patient_count]
train_val_patients = patient_ids[test_patient_count:]

test_df = all_data_df[all_data_df['patient_id'].isin(test_patients)].reset_index(drop=True)
train_val_df = all_data_df[all_data_df['patient_id'].isin(train_val_patients)].reset_index(drop=True)

print(f"Đã tách: {len(train_val_df)} mẫu cho Train/Validation (CV) và {len(test_df)} mẫu cho Test cuối cùng.")

le = LabelEncoder().fit(ALL_CLASSES)
sample_spec = np.load(train_val_df['filepath'][0])
INPUT_SHAPE = (sample_spec.shape[0], sample_spec.shape[1], 3)
print(f"Kích thước input được cập nhật: {INPUT_SHAPE}")

print("Đang fit StandardScaler...")
scaler_fit_sample_df = train_val_df.sample(n=min(len(train_val_df), 500), random_state=SEED)
scaler_fit_data = []
for filepath in scaler_fit_sample_df['filepath']:
    spec = np.load(filepath)
    spec_3_channels = np.stack([spec]*3, axis=-1)
    scaler_fit_data.append(spec_3_channels.flatten())
scaler = StandardScaler().fit(scaler_fit_data)
print("Fit StandardScaler hoàn tất.")

skf = StratifiedGroupKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
cv_data_to_split = train_val_df[train_val_df['label'].isin(CLASSES_TO_TRAIN)]
X_cv_paths = cv_data_to_split['filepath'].values
y_cv_labels = le.transform(cv_data_to_split['label'])
groups_cv = cv_data_to_split['patient_id'].values
fold_accuracies, fold_losses = [], []

AUTOTUNE = tf.data.AUTOTUNE

Bắt đầu chuẩn bị và phân chia dữ liệu...
Tách tập Test cuối cùng (Hold-out set)...
Đã tách: 28054 mẫu cho Train/Validation (CV) và 5030 mẫu cho Test cuối cùng.
Kích thước input được cập nhật: (256, 126, 3)
Đang fit StandardScaler...
Fit StandardScaler hoàn tất.


In [ ]:
tf.debugging.enable_check_numerics()
# VÒNG LẶP Cross-Validation
# try:
#     strategy = tf.distribute.MirroredStrategy()
#     print('Số lượng thiết bị: {}'.format(strategy.num_replicas_in_sync))
# except RuntimeError as e:
#     print(e)
#     # Nếu chỉ có 1 GPU hoặc không có GPU, dùng chiến lược mặc định
#     strategy = tf.distribute.get_strategy()

active_indices = [le.transform([c])[0] for c in CLASSES_TO_TRAIN]

for fold, (train_indices, val_indices) in enumerate(skf.split(X_cv_paths, y_cv_labels, groups_cv)):
    fold_number = fold + 1
    print("-" * 50 + f"\nBắt đầu Fold {fold_number}/{N_SPLITS}\n" + "-" * 50)
    
    train_paths, val_paths = X_cv_paths[train_indices], X_cv_paths[val_indices]
    train_labels, val_labels = y_cv_labels[train_indices], y_cv_labels[val_indices]
    
    train_labels_onehot = tf.keras.utils.to_categorical(train_labels, num_classes=len(ALL_CLASSES))
    val_labels_onehot = tf.keras.utils.to_categorical(val_labels, num_classes=len(ALL_CLASSES))
    
    # Tạo pipeline tf.data cho tập train
    train_ds = tf.data.Dataset.from_tensor_slices((train_paths, train_labels_onehot))
    train_ds = train_ds.map(parse_and_process, num_parallel_calls=AUTOTUNE)
    train_ds = train_ds.shuffle(buffer_size=SHUFFLE_BUFFER_SIZE)
    train_ds = train_ds.batch(BATCH_SIZE)
    if USE_DATA_AUGMENTATION:
        train_ds = train_ds.map(augment, num_parallel_calls=AUTOTUNE)
    train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)

    # Tạo pipeline tf.data cho tập validation
    val_ds = tf.data.Dataset.from_tensor_slices((val_paths, val_labels_onehot))
    val_ds = val_ds.map(parse_and_process, num_parallel_calls=AUTOTUNE)
    val_ds = val_ds.cache()
    val_ds = val_ds.batch(BATCH_SIZE)
    val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)

    #with strategy.scope():
    # Gọi create_model (giả định bạn đã sửa nó để lớp Dense cuối có activation='linear')
    model = create_model(INPUT_SHAPE, len(ALL_CLASSES))
    
    optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE, clipvalue=1.0)
        
    # --- THAY ĐỔI CHÍNH ĐỂ TĂNG CƯỜNG SỰ ỔN ĐỊNH ---
    loss_function = None
    if USE_FOCAL_LOSS:
        # Sử dụng hàm focal loss mới làm việc với logits
        loss_function = focal_loss_from_logits_optimized(active_indices=active_indices) 
    else:
        # Sử dụng CategoricalCrossentropy với from_logits=True
        loss_function = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
        
    model.compile(optimizer=optimizer, 
                loss=loss_function, 
                metrics=['accuracy'], 
                jit_compile=False)

    reduce_lr = ReduceLROnPlateau(monitor='val_loss', 
                            factor=0.5, 
                            patience=5, 
                            min_lr=1e-8, 
                            verbose=1)

    history = model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS, 
                        callbacks=[EarlyStopping(monitor='val_loss', patience=EARLY_STOPPING_PATIENCE, min_delta=MIN_DELTA, restore_best_weights=True),
                                    reduce_lr
                                  ],
                        verbose=1)
    
    # Code vẽ sơ đồ kết quả
    plt.figure(figsize=(15, 6))
    plt.suptitle(f'Training Metrics for Fold {fold_number}', fontsize=16)
    
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Accuracy vs. Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend(loc='lower right')
    plt.grid(True)
    
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Loss vs. Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(loc='upper right')
    plt.grid(True)
    
    plot_filename = f'fold_{fold_number}_metrics.png'
    plot_filepath = os.path.join(KAGGLE_OUTPUT_PATH, plot_filename)
    plt.savefig(plot_filepath)
    plt.close()
    
    print(f"Đã lưu biểu đồ cho Fold {fold_number} tại: {plot_filepath}")
    
    # Để evaluate mô hình có output là logits, không cần thay đổi gì ở đây
    loss, accuracy = model.evaluate(val_ds, verbose=0)
    print(f"Fold {fold_number} - Validation Loss: {loss:.4f}, Validation Accuracy: {accuracy:.4f}")
    fold_losses.append(loss)
    fold_accuracies.append(accuracy)

print("=" * 50 + "\nKết quả Cross-Validation:\n" + f"Validation Accuracy trung bình: {np.mean(fold_accuracies):.4f} +/- {np.std(fold_accuracies):.4f}\n" + f"Validation Loss trung bình: {np.mean(fold_losses):.4f} +/- {np.std(fold_losses):.4f}\n" + "=" * 50)

--------------------------------------------------
Bắt đầu Fold 1/5
--------------------------------------------------


I0000 00:00:1756218008.669381      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1756218008.670089      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


94668760/94668760 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 1/50


2025-08-26 14:20:14.565370: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_4}}
I0000 00:00:1756218102.428830     106 cuda_dnn.cc:529] Loaded cuDNN version 90300


351/351 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.8224 - loss: 0.0830

2025-08-26 14:30:44.824962: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_4}}


351/351 ━━━━━━━━━━━━━━━━━━━━ 728s 2s/step - accuracy: 0.8224 - loss: 0.0829 - val_accuracy: 0.4617 - val_loss: 0.4852 - learning_rate: 3.0000e-05
Epoch 2/50
351/351 ━━━━━━━━━━━━━━━━━━━━ 461s 1s/step - accuracy: 0.7488 - loss: 0.1198 - val_accuracy: 0.4886 - val_loss: 0.4735 - learning_rate: 3.0000e-05
Epoch 3/50
351/351 ━━━━━━━━━━━━━━━━━━━━ 457s 1s/step - accuracy: 0.7528 - loss: 0.1068 - val_accuracy: 0.5266 - val_loss: 0.3302 - learning_rate: 3.0000e-05
Epoch 4/50
351/351 ━━━━━━━━━━━━━━━━━━━━ 456s 1s/step - accuracy: 0.7591 - loss: 0.0994 - val_accuracy: 0.5383 - val_loss: 0.2772 - learning_rate: 3.0000e-05
Epoch 5/50
351/351 ━━━━━━━━━━━━━━━━━━━━ 464s 1s/step - accuracy: 0.7482 - loss: 0.0921 - val_accuracy: 0.5447 - val_loss: 0.2082 - learning_rate: 3.0000e-05
Epoch 6/50
351/351 ━━━━━━━━━━━━━━━━━━━━ 475s 1s/step - accuracy: 0.7569 - loss: 0.0948 - val_accuracy: 0.5410 - val_loss: 0.2592 - learning_rate: 3.0000e-05
Epoch 7/50
351/351 ━━━━━━━━━━━━━━━━━━━━ 461s 1s/step - accuracy: 0.75

In [ ]:
# HUẤN LUYỆN MÔ HÌNH CUỐI CÙNG
print("Bắt đầu huấn luyện lại mô hình cuối cùng trên toàn bộ dữ liệu Train+Validation...")
final_train_df = train_val_df[train_val_df['label'].isin(CLASSES_TO_TRAIN)]

# Tạo pipeline tf.data cho việc huấn luyện cuối cùng
final_train_paths = final_train_df['filepath'].values
final_train_labels = le.transform(final_train_df['label'])
final_train_labels_onehot = tf.keras.utils.to_categorical(final_train_labels, num_classes=len(ALL_CLASSES))

final_train_ds = tf.data.Dataset.from_tensor_slices((final_train_paths, final_train_labels_onehot))
final_train_ds = final_train_ds.map(parse_and_process, num_parallel_calls=AUTOTUNE)
final_train_ds = final_train_ds.shuffle(buffer_size=len(final_train_paths))
final_train_ds = final_train_ds.batch(BATCH_SIZE)
if USE_DATA_AUGMENTATION:
    final_train_ds = final_train_ds.map(augment, num_parallel_calls=AUTOTUNE)
final_train_ds = final_train_ds.prefetch(buffer_size=AUTOTUNE)


final_model = create_model(INPUT_SHAPE, len(ALL_CLASSES))

# SỬA LỖI 1: Sử dụng clipvalue để đảm bảo ổn định
final_optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE, clipvalue=1.0)

# SỬA LỖI 2: Sử dụng đúng hàm loss 'from_logits'
active_indices = [le.transform([c])[0] for c in CLASSES_TO_TRAIN]
loss_function = None
if USE_FOCAL_LOSS:
    loss_function = focal_loss_from_logits_optimized(active_indices=active_indices)
else:
    loss_function = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

final_model.compile(optimizer=final_optimizer, 
                    loss=loss_function, 
                    metrics=['accuracy'], 
                    jit_compile=False) # Tắt jit_compile để tránh các lỗi tiềm ẩn

run_timestamp = datetime.datetime.now(pytz.timezone('Asia/Ho_Chi_Minh')).strftime("%Y-%m-%d_%H-%M-%S")
model_checkpoint_path = os.path.join(KAGGLE_OUTPUT_PATH, f'{MODEL_ID}_final_model_{run_timestamp}.h5')

# GỢI Ý 3: Chỉ dùng ModelCheckpoint và huấn luyện đủ số epochs để mô hình hội tụ tốt
final_history = final_model.fit(final_train_ds, epochs=EPOCHS, 
                                callbacks=[ModelCheckpoint(filepath=model_checkpoint_path, 
                                                          save_best_only=True, 
                                                          monitor='loss',
                                                          verbose=1)], 
                                verbose=1)
print("Huấn luyện mô hình cuối cùng hoàn tất.")

In [ ]:
# ĐÁNH GIÁ MÔ HÌNH VÀ VẼ CÁC SƠ ĐỒ
print("\nĐang đánh giá mô hình cuối cùng trên tập Test (Hold-out)...")
final_model.load_weights(model_checkpoint_path)
final_test_df = test_df[test_df['label'].isin(CLASSES_TO_TRAIN)]



X_test, y_test_labels = load_data_from_df(final_test_df)
y_test_encoded = le.transform(y_test_labels)
y_test_onehot = tf.keras.utils.to_categorical(y_test_encoded, num_classes=len(ALL_CLASSES))
X_test = np.stack([X_test]*3, axis=-1)
X_test_flat = X_test.reshape(X_test.shape[0], -1)
X_test_scaled = scaler.transform(X_test_flat)
X_test = np.nan_to_num(X_test_scaled).reshape(X_test.shape)
print("Tải dữ liệu test hoàn tất!")

loss, accuracy = final_model.evaluate(X_test, y_test_onehot, verbose=0)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")

y_pred_probs = final_model.predict(X_test)
y_pred_encoded = np.argmax(y_pred_probs, axis=1)

trained_class_indices = np.unique(y_test_encoded)
target_names_trained = le.inverse_transform(trained_class_indices)

report = classification_report(y_test_encoded, y_pred_encoded, target_names=target_names_trained, labels=trained_class_indices)
print("\nClassification Report:\n", report)

report_figs_path = os.path.join(KAGGLE_OUTPUT_PATH, "report_figures")
os.makedirs(report_figs_path, exist_ok=True)

plt.figure(figsize=(8, 6))
plt.plot(final_history.history['accuracy'], label='Training Accuracy')
plt.title('Biểu đồ Accuracy của mô hình cuối cùng')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
accuracy_plot_path = os.path.join(report_figs_path, f'final_accuracy_plot_{run_timestamp}.png')
plt.savefig(accuracy_plot_path)
plt.close()

plt.figure(figsize=(8, 6))
plt.plot(final_history.history['loss'], label='Training Loss')
plt.title('Biểu đồ Loss của mô hình cuối cùng')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper right')
loss_plot_path = os.path.join(report_figs_path, f'final_loss_plot_{run_timestamp}.png')
plt.savefig(loss_plot_path)
plt.close()

cm = confusion_matrix(y_test_encoded, y_pred_encoded, labels=trained_class_indices)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=target_names_trained, yticklabels=target_names_trained)
plt.title('Ma trận nhầm lẫn trên tập Test cuối cùng')
plt.ylabel('Nhãn thật')
plt.xlabel('Nhãn dự đoán')
cm_plot_path = os.path.join(report_figs_path, f'confusion_matrix_{run_timestamp}.png')
plt.savefig(cm_plot_path)
plt.close()

In [ ]:
# VẼ GRAD-CAM
last_conv_layer_name = None
for layer in reversed(final_model.layers):
    if isinstance(layer, tf.keras.layers.GlobalAveragePooling2D):
        pooling_index = final_model.layers.index(layer)
        last_conv_layer_name = final_model.layers[pooling_index - 1].name
        break
if last_conv_layer_name is None:
    raise ValueError("Không thể tự động tìm thấy lớp phù hợp cho Grad-CAM.")
print(f"Đã tự động xác định lớp Grad-CAM: {last_conv_layer_name}")

gradcam_path = os.path.join(report_figs_path, "grad_cam")
os.makedirs(gradcam_path, exist_ok=True)
print("Tạo hình ảnh Grad-CAM...")
results_list = []
for i in range(len(y_test_encoded)):
    true_label_encoded = y_test_encoded[i]
    pred_label_encoded = y_pred_encoded[i]
    confidence = y_pred_probs[i][pred_label_encoded]
    is_correct = (true_label_encoded == pred_label_encoded)
    results_list.append({'index': i, 'true_label': true_label_encoded, 'pred_label': pred_label_encoded, 'confidence': confidence, 'is_correct': is_correct})
results_df = pd.DataFrame(results_list)

for class_index, class_name in zip(trained_class_indices, target_names_trained):
    correct_samples = results_df[(results_df['is_correct'] == True) & (results_df['true_label'] == class_index)].nlargest(3, 'confidence')
    incorrect_samples = results_df[(results_df['is_correct'] == False) & (results_df['true_label'] == class_index)].nlargest(3, 'confidence')
    for _, row in correct_samples.iterrows():
        idx = int(row['index'])
        img_array, spec = X_test[idx][np.newaxis, ...], X_test[idx, :, :, 0]
        heatmap = get_grad_cam(final_model, img_array, last_conv_layer_name, pred_index=class_index)
        overlay = overlay_grad_cam(spec, heatmap)
        plt.imshow(overlay)
        plt.title(f"Đúng: {class_name}, Tin cậy: {row['confidence']:.2f}")
        plt.axis('off')
        plt.savefig(os.path.join(gradcam_path, f"correct_{class_name}_{idx}_{run_timestamp}.png"))
        plt.close()
    for _, row in incorrect_samples.iterrows():
        idx = int(row['index'])
        pred_class_name = le.inverse_transform([int(row['pred_label'])])[0]
        img_array, spec = X_test[idx][np.newaxis, ...], X_test[idx, :, :, 0]
        heatmap = get_grad_cam(final_model, img_array, last_conv_layer_name, pred_index=class_index)
        overlay = overlay_grad_cam(spec, heatmap)
        plt.imshow(overlay)
        plt.title(f"Thật: {class_name}, Sai -> {pred_class_name}, Tin cậy: {row['confidence']:.2f}")
        plt.axis('off')
        plt.savefig(os.path.join(gradcam_path, f"incorrect_{class_name}_as_{pred_class_name}_{idx}_{run_timestamp}.png"))
        plt.close()

correct_heatmaps = {label: [] for label in target_names_trained}
incorrect_heatmaps = {label: [] for label in target_names_trained}
for i, row in tqdm(results_df.iterrows(), total=len(results_df), desc="Calculating Avg Grad-CAMs"):
    idx = int(row['index'])
    true_label_index = int(row['true_label'])
    class_name = le.inverse_transform([true_label_index])[0]
    img_array = X_test[idx][np.newaxis, ...]
    heatmap = get_grad_cam(final_model, img_array, last_conv_layer_name, pred_index=true_label_index)
    if row['is_correct']:
        if class_name in correct_heatmaps: correct_heatmaps[class_name].append(heatmap)
    else:
        if class_name in incorrect_heatmaps: incorrect_heatmaps[class_name].append(heatmap)

for class_name in target_names_trained:
    if correct_heatmaps.get(class_name):
        avg_heatmap_correct = np.mean(correct_heatmaps[class_name], axis=0)
        overlay = overlay_grad_cam(np.zeros(INPUT_SHAPE[:2]), avg_heatmap_correct)
        plt.imshow(overlay)
        plt.title(f"Grad-CAM TB - Đúng cho lớp {class_name}")
        plt.axis('off')
        plt.savefig(os.path.join(gradcam_path, f"avg_correct_{class_name}_{run_timestamp}.png"))
        plt.close()
    if incorrect_heatmaps.get(class_name):
        avg_heatmap_incorrect = np.mean(incorrect_heatmaps[class_name], axis=0)
        overlay = overlay_grad_cam(np.zeros(INPUT_SHAPE[:2]), avg_heatmap_incorrect)
        plt.imshow(overlay)
        plt.title(f"Grad-CAM TB - Sai cho lớp {class_name}")
        plt.axis('off')
        plt.savefig(os.path.join(gradcam_path, f"avg_incorrect_{class_name}_{run_timestamp}.png"))
        plt.close()

In [ ]:
# TẠO BÁO CÁO PDF
print("Tạo báo cáo PDF...")
pdf = PDFReport()
pdf.add_page()
pdf.chapter_title("1. Tom tat cau hinh va Ket qua")
config_summary = f"""
- Model ID: {MODEL_ID}
- Thoi gian chay: {datetime.datetime.now(pytz.timezone('Asia/Ho_Chi_Minh')).strftime("%Y-%m-%d %H:%M:%S")}
- Cac lop huan luyen: {', '.join(CLASSES_TO_TRAIN)}
- K-Fold Cross-Validation: {N_SPLITS} folds

--- KET QUA CROSS-VALIDATION ---
- Validation Accuracy trung binh: {np.mean(fold_accuracies):.4f} +/- {np.std(fold_accuracies):.4f}
- Validation Loss trung binh: {np.mean(fold_losses):.4f} +/- {np.std(fold_losses):.4f}

--- KET QUA TREN TAP TEST CUOI CUNG ---
- Test Loss: {loss:.4f}
- Test Accuracy: {accuracy:.4f}

--- CAU HINH CHI TIET ---
- SEED: {SEED}
- Epochs: {EPOCHS} (Patience: {EARLY_STOPPING_PATIENCE})
- Batch Size: {BATCH_SIZE}
- Learning Rate: {LEARNING_RATE}
- Ham Loss: {'Focal Loss' if USE_FOCAL_LOSS else 'Categorical Crossentropy'}
- Tang cuong du lieu: {'Co (SpecAugment)' if USE_DATA_AUGMENTATION else 'Khong'}
- Kich thuoc Input: {INPUT_SHAPE}
"""
pdf.chapter_body(config_summary)
pdf.add_image_section("2. Bieu do Huan luyen cua Mo hinh Cuoi cung", accuracy_plot_path)
pdf.add_image_section("", loss_plot_path)
pdf.chapter_title("3. Danh gia chi tiet tren tap Test")
pdf.chapter_body("Bao cao phan loai chi tiet:")
pdf.set_font('Courier', '', 8)
pdf.chapter_body(report)
pdf.add_image_section("Ma tran nham lan:", cm_plot_path)

pdf.add_page()
pdf.chapter_title("4. Phan tich Grad-CAM")
for class_name in target_names_trained:
    pdf.chapter_body(f"Lop: {class_name}")
    correct_imgs = sorted(glob.glob(os.path.join(gradcam_path, f"correct_{class_name}_*_{run_timestamp}.png")))
    incorrect_imgs = sorted(glob.glob(os.path.join(gradcam_path, f"incorrect_{class_name}_*_{run_timestamp}.png")))
    
    x_pos, y_pos = pdf.get_x(), pdf.get_y()
    for i, img_path in enumerate(correct_imgs[:3]):
        if os.path.exists(img_path): pdf.image(img_path, x=x_pos + i * 60, y=y_pos, w=55)
    if correct_imgs: y_pos += 45
    for i, img_path in enumerate(incorrect_imgs[:3]):
        if os.path.exists(img_path): pdf.image(img_path, x=x_pos + i * 60, y=y_pos, w=55)
    if incorrect_imgs: y_pos += 45
    pdf.set_y(y_pos)
    
    avg_correct_path = os.path.join(gradcam_path, f"avg_correct_{class_name}_{run_timestamp}.png")
    avg_incorrect_path = os.path.join(gradcam_path, f"avg_incorrect_{class_name}_{run_timestamp}.png")
    if os.path.exists(avg_correct_path):
        pdf.image(avg_correct_path, w=80)
    if os.path.exists(avg_incorrect_path):
        pdf.image(avg_incorrect_path, w=80)
    pdf.ln(10)

report_filename = f"report_{MODEL_ID}_{run_timestamp}.pdf"
report_filepath = os.path.join(KAGGLE_OUTPUT_PATH, report_filename)
pdf.output(report_filepath)
print(f"Đã tạo báo cáo PDF tại: {report_filepath}")

print("\nBắt đầu quá trình tải kết quả lên Google Drive...")
drive = authenticate_gdrive()
upload_folder_to_drive(drive, KAGGLE_OUTPUT_PATH, DRIVE_RESULTS_FOLDER_ID)
print("Hoàn tất! Toàn bộ kết quả đã được lưu về Google Drive.")